In [7]:
import numpy as np
import paddle
import paddle.nn as nn
import paddle.optimizer as opt

BATCH_SIZE = 16
BATCH_NUM = 4
EPOCH_NUM = 4

IMAGE_SIZE = 784
CLASS_NUM = 10

# define a random dataset
class RandomDataset(paddle.io.Dataset):
    def __init__(self, num_samples):
        self.num_samples = num_samples

    def __getitem__(self, idx):
        image = np.random.random([IMAGE_SIZE]).astype('float32')
        label = np.random.randint(0, CLASS_NUM - 1, (1, )).astype('int64')
        return image, label

    def __len__(self):
        return self.num_samples

class LinearNet(nn.Layer):
    def __init__(self):
        super(LinearNet, self).__init__()
        self._linear = nn.Linear(IMAGE_SIZE, CLASS_NUM)

    def forward(self, x):
        return self._linear(x)

def train(layer, loader, loss_fn, opt):
    for epoch_id in range(EPOCH_NUM):
        for batch_id, (image, label) in enumerate(loader()):
            out = layer(image)
            loss = loss_fn(out, label)
            loss.backward()
            opt.step()
            opt.clear_grad()
            print("Epoch {} batch {}: loss = {}".format(
                epoch_id, batch_id, np.mean(loss.numpy())))

# create network
layer = LinearNet()
loss_fn = nn.CrossEntropyLoss()
adam = opt.Adam(learning_rate=0.001, parameters=layer.parameters())

# create data loader
dataset = RandomDataset(BATCH_NUM * BATCH_SIZE)
loader = paddle.io.DataLoader(dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=2)

# train
train(layer, loader, loss_fn, adam)


Epoch 0 batch 0: loss = 2.9415159225463867
Epoch 0 batch 1: loss = 2.4194467067718506
Epoch 0 batch 2: loss = 2.3176610469818115
Epoch 0 batch 3: loss = 2.5523810386657715
Epoch 1 batch 0: loss = 2.217099189758301
Epoch 1 batch 1: loss = 2.6772921085357666
Epoch 1 batch 2: loss = 2.2865443229675293
Epoch 1 batch 3: loss = 2.4641644954681396
Epoch 2 batch 0: loss = 2.396103620529175
Epoch 2 batch 1: loss = 2.57643461227417
Epoch 2 batch 2: loss = 2.2850711345672607
Epoch 2 batch 3: loss = 2.4605910778045654
Epoch 3 batch 0: loss = 2.365332841873169
Epoch 3 batch 1: loss = 2.3683786392211914
Epoch 3 batch 2: loss = 2.3753390312194824
Epoch 3 batch 3: loss = 2.178776741027832


d:\tool\python36\lib\site-packages\paddle\fluid\reader.py:352: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  "DataLoader with multi-process mode is not supported on MacOs and Windows currently." \


# save分别保存net和opt参数

In [8]:
# save
paddle.save(layer.state_dict(), "linear_net.pdparams")
paddle.save(adam.state_dict(), "adam.pdopt")


In [9]:
layer_state_dict = paddle.load("linear_net.pdparams")
opt_state_dict = paddle.load("adam.pdopt")

layer.set_state_dict(layer_state_dict)
adam.set_state_dict(opt_state_dict)

In [ ]:
# 包装为Model保存

In [10]:
model=paddle.Model(layer)
model.save('model') #会自动加.pdparams


In [12]:
# paddle2.3可运行
obj_data={'model':layer.state_dict(),'opt':adam.state_dict()}
paddle.save(obj_data,'model_opt')

TypeError: can't pickle ParamBase objects